![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials:

# Ports

## What is a `Port`?!

A `Port` object is a collection of variables that help basic use: check compatibility, plug, etc.
Without this object, connections between `Systems` should be done variable-by-variable, *outch!*

![Connected ports](images/ports_1.svg)

## Types

### `IN` or `OUT`

Ports are directional objects, they can be either `IN` or `OUT`. This definition ensure compatibility at connection.

### Available ports and compatibility

<font color='orange'>**CoSApp**</font> allow user-defined ports, stored in libraries or created on-the-fly. They allow a frozen collection of variables that can be shared between multiple systems. They are compatible by construction (see [Ports connection](#Ports-connection) section of this tutorial).

## Create a port

### Import CoSApp core package

In [ ]:
# import cosapp or cosapp base classes depending your usage
from cosapp.systems import System
from cosapp.ports import Port

#### Define a new port

In [ ]:
class DemoPort(Port):
    
    def setup(self):
        self.add_variable('a', 1)
        self.add_variable('b', 2)
        self.add_variable('c', 3)

#### Use it in a system

Create a new `System` where the `DemoPort` is used

In [ ]:
class DemoSystem(System):

    def setup(self):
        self.add_input(DemoPort, 'p_in')
        self.add_output(DemoPort, 'p_out') 

    def compute(self):
        self.p_out.a = self.p_in.c
        self.p_out.b = self.p_in.b
        self.p_out.c = self.p_in.a

s = DemoSystem(name='demo')

![Port-DemoSystem](images/ports_2.svg)

Run the system to confirm the expected behaviour

In [ ]:
s.run_once()
print('s.p_in')
s.p_in

In [ ]:
print('s.p_out')
s.p_out

#### Set information on the variables

All variables in **CoSApp** accept some additional informations:

- *unit*: Physical unit of the variable - given by a string. Units *are not enforced* inside a `System`.
This means that the user must ensure the computed variables in method `compute` are converted to the user
specified unit set in `setup`. **CoSApp** will take care of unit conversion between connected `System`s.
- *desc*: Short description of the variable.
- *dtype*: If you need to force certain data type(s) on a variable, a tuple of acceptable types can be provided
through this keyword. If that information is not supplied, dtype is inferred from the variable value; e.g.
a number (integer or floating point) will be typed as `Number`.

In [ ]:
class AdvancedDemoPort(Port):
    
    def setup(self):
        self.add_variable('a', 1., unit='degK', dtype=float, desc='Temperature')
        self.add_variable('b', 2, unit='Pa', dtype=(int, float), desc='Pressure')
        self.add_variable('c', 3, unit='kg/s', desc='Mass flow')

class AdvancedDemoSystem(System):

    def setup(self):
        self.add_input(AdvancedDemoPort, 'p_in')
        self.add_output(AdvancedDemoPort, 'p_out') 

    def compute(self):
        self.p_out.a = self.p_in.c
        self.p_out.b = self.p_in.b
        self.p_out.c = self.p_in.a

advanced_s = AdvancedDemoSystem(name='demo')
print('Output port')
advanced_s.p_out

## Ports connection

### Introduction

Ports connection consists to create a reference in a `Port` to another to allow the transfer of information between systems when needed. The object used to connect 2 ports is called a `Connector`.

![Port-Connection](images/ports_3.svg)

If you want to connect all port variables at once, use the `connect` method of `Port`:

In [ ]:
h = System('head')
h.add_child(DemoSystem('demo1'))
h.add_child(DemoSystem('demo2'))

h.connect(h.demo2.p_in, h.demo1.p_out)

If you need to connect a subset of the `Port` or connect to some inwards or outwards, use the `connect` method of `Port`:

In [ ]:
class MonitorSystem(System):

    def setup(self):
        self.add_inward('a')
        self.add_outward('result')

    def compute(self):
        self.result = self.a

h.add_child(MonitorSystem('monitor1'))
h.connect(h.monitor1.inwards, h.demo1.p_out, 'a')

h.demo1.p_in.a = 50.
h.demo1.p_in.c = 25.
h.run_drivers()

print('h.demo1.p_in')
h.demo1.p_in

In [ ]:
print('h.demo2.p_in')
h.demo2.p_in

In [ ]:
print('h.monitor1.outwards')
h.monitor1.outwards

**Congrats!** Now you have the basic of `Port` in **CoSApp**

Next you will discover how to solve mathematical problem using [Drivers](03-Drivers.ipynb).